## Fooling around to check what's going on

In [46]:
from pymongo import MongoClient
import pandas as pd
import json as js

In [42]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [43]:
# looking for companies

## Haz las queries necesarias, como raised money, y coges esa columna y la pegas a este dataset de first_data, ( y que tengas en la query nombre y la fila que quieras, para que te lo indexe)

In [48]:
db, coll = connectCollection('companies','companies')

In [51]:
rest = list(coll.find())

In [55]:
first_data=pd.DataFrame(rest)
first_data

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,screenshots,external_links,partners,founded_month,founded_day,location,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,"{'type': 'Point', 'coordinates': [-122.333253,...",NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN,NaN
3,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,[],[],[],11.0,1.0,NaN,9.0,15.0,,None
4,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],[],[],6.0,2.0,"{'type': 'Point', 'coordinates': [-122.247573,...",NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,entegreat-solutions,http://www.crunchbase.com/company/entegreat-so...,,,,,software,NaN,...,[],[],[],NaN,NaN,NaN,NaN,NaN,None,None
18797,52cdef7f4bab8bd67529c6f8,goBookmaker,gobookmaker,http://www.crunchbase.com/company/gobookmaker,http://www.gobookmaker.com,http://blog.gobookmaker.com,,gobookmaker,web,NaN,...,"[{'available_sizes': [[[150, 80], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,None,None
18798,52cdef7f4bab8bd67529c6f7,AfterLogic,afterlogic,http://www.crunchbase.com/company/afterlogic,http://www.afterlogic.com,,,afterlogic,software,NaN,...,"[{'available_sizes': [[[150, 137], 'assets/ima...",[],[],NaN,NaN,NaN,NaN,NaN,None,None
18799,52cdef7f4bab8bd67529c6fa,EnteGreat,entegreat,http://www.crunchbase.com/company/entegreat,http://www.entegreat.com,,,,enterprise,NaN,...,[],[],[],NaN,NaN,NaN,NaN,NaN,None,None


In [57]:
first_data.shape

(18801, 43)

In [9]:
print(len(rest))

3787


In [13]:
# we found latitude and longitude
rest[0]['offices']

[{'description': '',
  'address1': '710 - 2nd Avenue',
  'address2': 'Suite 1100',
  'zip_code': '98104',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.603122,
  'longitude': -122.333253},
 {'description': '',
  'address1': '270 Lafayette Street',
  'address2': 'Suite 505',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'latitude': 40.7237306,
  'longitude': -73.9964312}]

## GeoJSON

In [19]:
# mongo kind of sucks because this function depends on how was written the entry of a
# place. it could be in a different position than mentioned below, and we're doomed.
# i need to sharp this function a lot
def getLocation(place):
    longitude = place['offices'][0]["longitude"]
    latitude = place['offices'][0]["latitude"]
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [20]:
getLocation(rest[0])

{'type': 'Point', 'coordinates': [-122.333253, 47.603122]}

In [32]:

#si quiero añadir información a la colección $set (para el geoindex)

for place in rest:
    value = {"$set": {'location':getLocation(place)}}
    coll.update_one(place,value)
    # <collection>.update_one(<do

In [34]:
# Supongo que de la misma manera unset la elimina (mira bien esto)
for place in rest:
    value = {"$unset": {'address.coord':''}}
    coll.update_one(place,value)
    # <collection>.update_one(<document>,<value>)


In [25]:
rest = list(coll.find({'name':"Amazon"}))
list(rest[0])

['_id',
 'name',
 'permalink',
 'crunchbase_url',
 'homepage_url',
 'blog_url',
 'blog_feed_url',
 'twitter_username',
 'category_code',
 'number_of_employees',
 'founded_year',
 'founded_month',
 'founded_day',
 'deadpooled_year',
 'deadpooled_month',
 'deadpooled_day',
 'deadpooled_url',
 'tag_list',
 'alias_list',
 'email_address',
 'phone_number',
 'description',
 'created_at',
 'updated_at',
 'overview',
 'image',
 'products',
 'relationships',
 'competitions',
 'providerships',
 'total_money_raised',
 'funding_rounds',
 'investments',
 'acquisition',
 'acquisitions',
 'offices',
 'milestones',
 'ipo',
 'video_embeds',
 'screenshots',
 'external_links',
 'partners']

In [38]:
# para encontrar todo lo que hay en un radio (tienes que introducir las coord)
berlin_radio = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[122.3321, 47.6062]
       },
       "$maxDistance":10000
      }
     }
    }
)
berlin_radio=list(berlin_radio)

OperationFailure: error processing query: ns=companies.companiesTree: GEONEAR  field=location maxdist=10000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query

In [31]:
import foli

OperationFailure: error processing query: ns=companies.companiesTree: GEONEAR  field=location maxdist=10000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query